# Deploy R Shiny

### Introduction

This notebook shows how the R Shiny Application can be deployed. 

The notebook we will:
* Store Shiny assets into the same deployment space.
* Deploy Shiny assets as an app and view the dashboard.

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020, 2021. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp. <br>**

In [22]:
import os
import pandas as pd
import datetime
from ibm_watson_machine_learning import APIClient

token = os.environ['USER_ACCESS_TOKEN']
url= os.environ['RUNTIME_ENV_APSX_URL']

wml_credentials = {
   "token": token,
   "instance_id" : "openshift",
   "url": url,
   "version": "3.5"
}

client = APIClient(wml_credentials) 

### Set up Deployment Space

The following code programmatically gets the deployment Space or creates one if one doesn't already exist.  The Space name can be controlled through the variable *SPACE_NAME*. 

In [23]:
SPACE_NAME = 'Supply Chain'

# Check is the space already exists
space_uid = ''
for space in client.spaces.get_details()['resources']:
    if space['entity']['name'] == SPACE_NAME:
        space_uid=space['metadata']['id']
        client.set.default_space(space_uid)
        break

if space_uid == '':
    print("Deployment space",SPACE_NAME,"not found.")

# set this space as default space
client.set.default_space(space_uid)

'SUCCESS'

### User Input

Specify below the hostname to be used in the RShiny deployment

In [24]:
hostname = "<replace with the hostname e.g. 'cpd-cluster.company.com' >"

# Deploy Shiny App

In this section we will complete the steps to deploy a Shiny Dashboard in Cloud Pak for Data. The app can be deployed in a similar way to models and functions, using the watson_machine_learning_client package.

All of the files associated with the dashboard are contained in a zip file which is stored in data assets. If the user would like to make changes to the dashboard, they can download the zip from data assets and upload it in the RStudio IDE. 

In [25]:
r_shiny_deployment_name='supply_chain_Shiny_App'

### Store the App

Create the associated metadata and store the dashboard zip file in the deployment space. 

In [26]:
# Meta_props to store assets in space 
meta_props = {
    client.shiny.ConfigurationMetaNames.NAME: "Supply_Chain_Shiny_assets",
    client.shiny.ConfigurationMetaNames.DESCRIPTION: 'Store shiny assets in deployment space' # optional
}
app_details = client.shiny.store(meta_props, '/project_data/data_asset/SupplyChain-sample-application.zip')

Creating Shiny asset...
SUCCESS


### Deploy the App

Create the metadata for the Shiny deployment by providing  name, description, R-Shiny options and Hardware specifications. R-Shiny configuration provides options on whom you want to share the dashboard with, they are 
* Anyone with the link 
* Authenticated users  
* Collaborators in this deployment space

In [27]:
# Deployment metadata.
deployment_meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: r_shiny_deployment_name,
    client.deployments.ConfigurationMetaNames.DESCRIPTION: 'Deploy Supply Chain sample application',
    client.deployments.ConfigurationMetaNames.R_SHINY: { 'authentication': 'anyone_with_url' },
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: { 'name': 'S', 'num_nodes': 1}
}

# Create the deployment.
app_uid = client.shiny.get_uid(app_details)
rshiny_deployment = client.deployments.create(app_uid, deployment_meta_props)



#######################################################################################

Synchronous deployment creation for uid: 'c163e848-378e-443b-add5-29ed1d5ca126' started

#######################################################################################


initializing........
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='ee5bb7d6-3b0b-4d18-a1f5-eb767a931d9e'
------------------------------------------------------------------------------------------------




### Launch Shiny App
Now that the dashboard is deployed, it can be accessed through the web browser. The app URL can be found by navigating to the deployed app in the deployment space. 

Open the Navigation Menu, select **Deployments -> Spaces -> life_event_space -> Deployments -> life_event_prediction_Shiny_App** to find the dashboard URL.

Alternatively, the path for the app URL can be found from the deployment metadata created in the previous cell. This path should be appended to the user's Cloud Pak for Data hostname to get the complete app URL. To get the path, run the cell below:

In [28]:
print("https://"+hostname+"/ml/v4/deployments/"+rshiny_deployment['metadata']['id'] + '/r_shiny')

https://dse-cpd35-cluster1.datascienceelite.com/ml/v4/deployments/ee5bb7d6-3b0b-4d18-a1f5-eb767a931d9e/r_shiny
